# Model Preparation for Quantization

Torch에서는 Quantization이 Module by Module basis로 동작하기 때문에 아래와 같은 부수적인 것들은 준비해야 된다.

1. Output requantization이 필요한 연산들에 대해서 funcational에서 `nn.module` 형태로 변환
2. 어느 특정 레이어에 대해서 Quantized를 하려면 `.qconfig` 값을 별도로 세팅

또한 아래의 PTQ, QAT와 같은 static quantization을 적용하려면 추가적인 준비가 더 필요하다.
1. `QuantStub`, `DeQuantStub`이 모듈의 앞 뒤에 붙어야 함
2. Quantization을 위한 특수한 핸들링(add or cat)을 수행할 때 `FloatFunctional` 사용이 요구 됨
3. Layer Fusion을 위한 `Fuse modules` 사용

먼저 Quantization이 가능한 단순한 floating point model을 정의하면 다음과 같다.<br>
Quant, De-Quant가 layer를 감싸는 형식으로 구성해야하며, 가끔 연산이 수행되지 않는 layer의 경우 해당 layer를 제외하고 2개의 part로 quant-dequant로 묶어야한다.


In [3]:
import torch

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.quant = torch.quantization.QuantStub() # floating point tensor를 quantized tensor로 변환

        self.conv = torch.nn.Conv2d(3, 3, 1)
        self.relu = torch.nn.ReLU()
        self.flatten = torch.nn.Flatten()

        self.linear = torch.nn.Linear(3 * 32 * 32, 10)

        self.dequant = torch.quantization.DeQuantStub() #quantized tensor에서 floating point로 변환

    def forward(self, x):
        x = self.quant(x) #모델의 앞에 Quant 추가 
        x = self.conv(x)
        x = self.relu(x)

        x = self.flatten(x)
        x = self.linear(x)
        x = self.dequant(x) #모델의 뒤에 De-Quant 추가

        return x

Quantization Approach는 크게 2가지로 나뉜다.

# 1. PTQ : Post Training Quantization

PTQ는 학습 후에 quantization parameter (scale, shift)를 결정한다.

- **Dynamic range quantization**(weight only quantization)

weight만 (일반적으로)8-bit로 quantize된다. 따라서 모델 용량은 1/4 정도 감소한다. 또한 별도의 calibration 데이터가 필요하지 않다.<br>
inference 시에는 floating-point로변환되어 수행되므로 CPU 상의 속도 향상은 미미하다.


- **Full integer quantization**(weight and activation quantization)

weight 뿐만 아니라 모델의 입력 데이터, activations(중간 레이어의 output)들 또한 8-bit로 quantize<br>
더 적은 메모리 사용량, Cache 재사용성 증가라는 장점이 있다.<br>
하지만 Activations의 parameter를 결정하기 위한 calibration 데이터가 필요하다.<br>


- TensorRT의 calibration<br>

TensorRT는 zero point를 사용하지않는 Symmetric Quantization을 수행한다.<br>
이때 calibration은 성능 저하를 최소로하는 `threshold` 및 `scale`을 찾게된다. <br>
각 레이어 마다 activation value의 범위와 분포는 모두 다르다.<br>
이때 특정 threshold에서 saturated된 normalized histogram 분포(ref_distr(P))와
원 histogram로부터 quantized, normalized된 분포(quant_distr(Q))의
KL divergence를 측정하고 최소인 지점을 threshold로 설정한다.<br>


- **Float16 quantization**

fp32의 데이터 타입의 weight를 fp16으로 quantize<br>
모델 용량이 1/2 줄어들고 성능 저하가 적다. 또한 GPU 상에서 빠른 연산이 가능하다.<br>
하지만 반대로 CPU 상에서는 fixed point 연산만큼의 속도 향상이 있지는 않다.<br>

다음과 같이 Inference시에 `half()`를 통해서 입력 데이터와 모델의 dtype을 fp16으로 간단히 변환할 수 있다. <br>
학습 과정에서는 Mixed precision 기능을 활용하면된다.


In [ ]:
import torch

@torch.no_grad()
def test_model(model, testloader, device, half=False):
    correct = 0
    total = 0
    model = model.to(device)

    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        if half:
            images = images.half()
        outputs = model(images)
        _, predicted = torch.max(outputs, axis=1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()

model = load_model("fp32_path")
model.eval()
fp16_model = model.half()
test_model(model=fp16_model,
           testloader=testloader,
           device=device,
           half=True)

## PTQ 적용 과정

1. Quantization configuration<br>
: 어떤 하드웨어를 사용하냐에 따라서 다른 backend값 설정한다.

In [ ]:
backend = "fbgemm"
model_fp32.qconfig = torch.quantization.get_default_config(backend)​

2. Layer Fusion 수행: Fuse & Prepare<br>

In [ ]:
model_fp32_fused = torch.quantization.fuse_modules(model_fp32, [['conv', 'relu']])
model_fp32_prepared = torch.quantization.prepare(model_fp32_fused)

3. Calibration 수행

In [ ]:
# calibration with Train Data
test_model(model_fp32_prepared, trainloader, 'cpu')

4. Convert 수행 (int8 형태로 변환)

In [ ]:
model_int8 = torch.quantization.convert(model_fp32_prepared)

# 2. QTA : Quantization Aware Training

QAT는 학습 시점에 quantization을 emulate 하여 , 추론 시에 발생하는 quantization 오류를 학습 시점에 반영가능하도록 하는 방법<br>
PTQ 대비 quantization으로 인한 성능 하락 폭이 적은 것이 큰 장점이다.

학습에서 back propagation을 적용하기 위해 floating point로 변하게 된다. 이때 y=x 모양의 형태로 변해서 gradient를 linear(=1)로 가정하여 네트워크 학습을 수행한다.<br>





## QAT 적용 과정

PTQ 적용하는 과정이랑 거의 비슷하다.

In [ ]:
# 하드웨어에 따른 backend 설정
backend = "fbgemm"
model_fp32.qconfig = torch.quantization.get_default_config(backend)

# layer fusion & prepare
model_fp32_fused = torch.quantization.fuse_modules(model_fp32, [['conv', 'relu']])
model_fp32_prepared = torch.quantization.prepare_qat(model_fp32_fused)

# calibration
train(model_fp32_prepared)

# conver into int-8
model_int8_qat = torch.quantization.convert(model_fp32_prepared)